In [1]:
import pandas as pd
import numpy as np

In [2]:
df_ratings = pd.read_csv("./data/ratings_small.csv")
df_metadata = pd.read_csv("./data/movies_metadata.csv")
df_credits = pd.read_csv("./data/credits.csv")
df_keywords = pd.read_csv("./data/keywords.csv")

In [3]:
df_metadata['id'] = pd.to_numeric(df_metadata['id'], errors='coerce', downcast='integer')
df_metadata['id'][0]

862.0

In [4]:
df_keywords['id'][1]

8844

In [5]:
df_movies = pd.merge(df_metadata, df_credits, how='outer', on='id')
df_movies = pd.merge(df_movies, df_keywords, how='outer', on='id')
df_movies.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew,keywords
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862.0,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...","[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."


In [6]:
movies_columns= ['adult', 'budget', 'genres', 'id', 'original_language',
                'popularity', 'production_countries', 'revenue', 'runtime',
                'spoken_languages', 'status', 'video', 'vote_average', 'vote_count']

df_movies = df_movies[movies_columns]

df_movies['video']= np.where(df_movies['video']==True, 1, 0)
df_movies['adult']= np.where(df_movies['adult']==True, 1, 0)

df_movies.head(1)

,adult,budget,genres,id,original_language,popularity,production_countries,revenue,runtime,spoken_languages,status,video,vote_average,vote_count
0,0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862.0,en,21.9469,"[{'iso_3166_1': 'US', 'name': 'United States o...",373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,0,7.7,5415.0


In [7]:
import ast
def modify_prod_country(x):
    try:
        country= [i['iso_3166_1'] for i in ast.literal_eval(x)][0]
    except:
        return 'NaN'
    return country

df_movies['production_countries'] = df_movies['production_countries'].apply(modify_prod_country)

def modify_language(x):
    try:
        lang= [i['iso_639_1'] for i in ast.literal_eval(x)][0]
    except:
        return 'NaN'
    return lang

df_movies['spoken_languages'] = df_movies['spoken_languages'].apply(modify_language)

df_movies.head(1)

,adult,budget,genres,id,original_language,popularity,production_countries,revenue,runtime,spoken_languages,status,video,vote_average,vote_count
0,0,30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862.0,en,21.9469,US,373554033.0,81.0,en,Released,0,7.7,5415.0


In [8]:
### genres
def modify(x):
    key = [i['name'] for i in ast.literal_eval(x)]
    return key

df_movies['genre_modified'] = df_movies['genres'].apply(modify)

all_genre = []
for i in df_movies['genre_modified']:
    for j in i:
        all_genre.append(j)
        
new_genre_cols = list(set(all_genre))

for col in new_genre_cols:
    df_movies[col] = 0
    
for i in new_genre_cols:
    df_movies.loc[df_movies['genre_modified'].apply(lambda x: True if i in x else False), i]=1
    
df_movies.drop(['genres', 'genre_modified'], axis=1, inplace= True)

In [9]:
df_movies.head(1)

,adult,budget,id,original_language,popularity,production_countries,revenue,runtime,spoken_languages,status,...,Documentary,Adventure,Music,Vision View Entertainment,Foreign,Science Fiction,The Cartel,Action,Aniplex,Rogue State
0,0,30000000,862.0,en,21.9469,US,373554033.0,81.0,en,Released,...,0,0,0,0,0,0,0,0,0,0


In [10]:
df_movies.size

2098440

In [11]:
df_movies.dropna(inplace=True, axis=0)
df_movies.size

2082240

In [12]:
del df_metadata, df_keywords, df_credits

In [13]:
df_ratings.head(1)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144


In [14]:
df_movies = df_movies.rename(columns={'id':'movieId'})
df_ratings.drop(['timestamp'], axis=1, inplace=True)

In [15]:
df_movies['movieId'] = df_movies['movieId'].astype(int)
df_ratings['movieId'] = df_ratings['movieId'].astype(int)

In [16]:
df_final = pd.merge(df_movies, df_ratings, on=['movieId'])
df_final.head(1)

,adult,budget,movieId,original_language,popularity,production_countries,revenue,runtime,spoken_languages,status,...,Music,Vision View Entertainment,Foreign,Science Fiction,The Cartel,Action,Aniplex,Rogue State,userId,rating
0,0,60000000,949,en,17.9249,US,187436818.0,170.0,en,Released,...,0,0,0,0,0,1,0,0,23,3.5


In [17]:
del df_movies, df_ratings

In [18]:
from sklearn.preprocessing import LabelEncoder

char_cols= ['userId', 'original_language', 'production_countries', 'spoken_languages', 'status', 'video']
for col in char_cols:
    le = LabelEncoder()
    le.fit(list(df_final[col]))
    df_final[col]= le.transform(list(df_final[col]))
df_final.head(1)

,adult,budget,movieId,original_language,popularity,production_countries,revenue,runtime,spoken_languages,status,...,Music,Vision View Entertainment,Foreign,Science Fiction,The Cartel,Action,Aniplex,Rogue State,userId,rating
0,0,60000000,949,6,17.9249,59,187436818.0,170.0,15,2,...,0,0,0,0,0,1,0,0,22,3.5


In [19]:
df_final.to_csv('movie_data.csv', index=False)

In [20]:
data = pd.read_csv('movie_data.csv')
data.head(1)

,adult,budget,movieId,original_language,popularity,production_countries,revenue,runtime,spoken_languages,status,...,Music,Vision View Entertainment,Foreign,Science Fiction,The Cartel,Action,Aniplex,Rogue State,userId,rating
0,0,60000000,949,6,17.924927,59,187436818.0,170.0,15,2,...,0,0,0,0,0,1,0,0,22,3.5
